# Setting up the DB

This script should be run once in order to create the database.

In [1]:
from sqlalchemy import create_engine, Table
from sqlalchemy import Column, Integer, String, Text, Date, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from datetime import datetime

In [2]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [3]:
engine = create_engine('sqlite:///crm.db', echo=False)
Base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()

In [11]:
class MessageAlreadySent(Exception):
    pass

In [4]:
class Customer(Base):
    __tablename__ = 'customers'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    last_name = Column(String)
    email = Column(String)
    
    def create_msg(self, message_name, template_file):
        message_count = session.query(Message).filter_by(name=message_name, customer=self).count()
        if message_count > 0:
            raise MessageAlreadySent('Message {} already sent to {}'.format(message_name, self.email))
        with open(template_file, 'r') as template:
            template = template.read()
            text = template.format(name=self.name)
        message = Message(name=message_name, text=text, customer=self, date=datetime.now())
        return message

    def __repr__(self):
        return "<Customer(name='{}', last_name='{}', email='{}')>".format(
        self.name, self.last_name, self.email)

In [5]:
class Message(Base):
    __tablename__ = 'messages'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    text = Column(Text)
    date = Column(Date)
    
    customer_id = Column(Integer, ForeignKey('customers.id'))
    customer = relationship("Customer", back_populates="messages")
    
    def send(self, config):
        me = config['me']
        you = '{} <{}>'.format(self.customer.name, self.customer.email) 
        msg = MIMEMultipart('alternative')
        msg['From'] = me
        msg['To'] = you
        msg['Subject'] = self.name
        msg.attach(MIMEText(self.text, 'plain'))
        with smtplib.SMTP(config['EMAIL']['smtp_server'], config['EMAIL']['port']) as s:
            s.ehlo()
            s.login(config['EMAIL']['username'],config['EMAIL']['password'])
            s.sendmail(me, you, msg.as_string())

    def __repr__(self):
        return "<Message(name='{}', date='{}', customer='{}')>".format(
            self.name, self.date, self.customer)

In [6]:
class List(Base):
    __tablename__ = 'lists'
    id = Column(Integer, primary_key=True)
    name = Column(String)

In [7]:
association_table = Table('list_customer', Base.metadata,
    Column('left_id', Integer, ForeignKey('customers.id')),
    Column('right_id', Integer, ForeignKey('lists.id'))
)

In [8]:
Customer.messages = relationship('Message', order_by=Message.id, back_populates='customer')
Customer.lists = relationship("List",
                    secondary=association_table,
                    backref="customers")

In [9]:
Base.metadata.create_all(engine)